# CNN for sample data

In [1]:
%pylab
%matplotlib inline

import glob
import os


CORPORA_PATH = "~/projects/corpora/sets/sample"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))




Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%%capture
from helpers import load_multiple_data
X, y = load_multiple_data(files)

Agrego una dimensión dummy ya que `keras` va a tratar esto como una imagen

In [3]:
X_t = X[:, :, :, np.newaxis]

X_t.shape

(42480, 14, 104, 1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_t, y, test_size=0.1, stratify=y)

X_train.shape, y_train.shape

((38232, 14, 104, 1), (38232, 1))

## Función de activación sigmoidea
No estoy muy seguro del porqué ocurre esto, pero la función de activación sigmoidea pareciera estar overfitteando. Esto no ocurre cuando uso reLU.

In [11]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout, MaxPool2D



def build_model(activation_function='relu', dropout_probs=[0.55], n_kernels=5):
    model = Sequential()
    
    model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                    activation=activation_function, input_shape=(14, 104, 1)))
    model.add(Conv2D(3*n_kernels, (1, 10), padding='same',
                    activation=activation_function))
    model.add(Flatten())
    if len(dropout_probs) > 0:
        model.add(Dropout(dropout_probs[0]))
    model.add(Dense(32, activation=activation_function))
    if len(dropout_probs) > 1:
        model.add(Dropout(dropout_probs[1]))
    model.add(Dense(1, activation='sigmoid'))

    return model

model = build_model(activation_function='sigmoid')

model.compile(
  loss='binary_crossentropy', # using the cross-entropy loss function
  optimizer='rmsprop', 
  metrics=['accuracy']) # reporting the accuracy


In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint(filepath='sample_model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

model.fit(
    X_train, y_train, epochs=3,
    batch_size=64, class_weight={0:1, 1:6}, validation_split=0.05,
    callbacks=[checkpointer, early_stopping]
)

Train on 36320 samples, validate on 1912 samples
Epoch 1/3
36320/36320 [==============================] - 39s 1ms/step - loss: 1.5220 - acc: 0.1676 - val_loss: 1.2594 - val_acc: 0.1647
Epoch 2/3
36320/36320 [==============================] - 39s 1ms/step - loss: 1.2641 - acc: 0.1668 - val_loss: 1.2599 - val_acc: 0.1647
Epoch 3/3
36320/36320 [==============================] - 37s 1ms/step - loss: 1.2642 - acc: 0.1668 - val_loss: 1.2580 - val_acc: 0.1647


In [13]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.16666666666666666
Precision  = 0.16666666666666666
Recall     = 1.0
ROC AUC    = 0.5



## Usando reLU

In [14]:
model = build_model(activation_function='relu')

model.compile(
  loss='binary_crossentropy', # using the cross-entropy loss function
  optimizer='rmsprop', 
  metrics=['accuracy']) # reporting the accuracy

In [15]:
checkpointer = ModelCheckpoint(filepath='sample_model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

model.fit(
    X_train, y_train, epochs=3,
    batch_size=64, class_weight={0:1, 1:6}, validation_split=0.05,
    callbacks=[checkpointer, early_stopping]
)

Train on 36320 samples, validate on 1912 samples
Epoch 1/3
36320/36320 [==============================] - 39s 1ms/step - loss: 1.2824 - acc: 0.2588 - val_loss: 1.2570 - val_acc: 0.1763
Epoch 2/3
36320/36320 [==============================] - 38s 1ms/step - loss: 1.2648 - acc: 0.2605 - val_loss: 1.2537 - val_acc: 0.3687
Epoch 3/3
36320/36320 [==============================] - 41s 1ms/step - loss: 1.2589 - acc: 0.3216 - val_loss: 1.2528 - val_acc: 0.2312


In [16]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.2245762711864407
Precision  = 0.16914022517911975
Recall     = 0.9336158192090396
ROC AUC    = 0.5694113281624055

